# Проект: "Анализ поведения пользователей мобильного приложения"

**Описание проекта:** 

Анализируем поведение пользователей мобильного приложения по продаже продуктов питания для стартапа.

**Цели и задачи:**

* Изучить воронку продаж. Узнать, как пользователи доходят до покупки. Сколько пользователей доходит до покупки, а сколько — «застревает» на предыдущих шагах? На каких именно?
* Исследовать результаты A/A/B-эксперимента. Дизайнеры захотели поменять шрифты во всём приложении, а менеджеры испугались, что пользователям будет непривычно. Договорились принять решение по результатам A/A/B-теста. Пользователей разбили на 3 группы: 2 контрольные со старыми шрифтами и одну экспериментальную — с новыми. Выясним, какой шрифт лучше.

**Описание данных:**

Каждая запись в логе — это действие пользователя, или событие. 
* `EventName` — название события;
* `DeviceIDHash` — уникальный идентификатор пользователя;
* `EventTimestamp` — время события;
* `ExpId` — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.

**План исследования:**

* 1. Откроем файл с данными и изучим общую информацию
* 2. Подготовим данные
* 3. Изучим и проверим данные
* 4. Изучим воронку событий
* 5. Изучим результаты эксперимента

## Загрузим данные и подготовим их к анализу

In [3]:
# Загружаем библиотеки
import pandas as pd
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import seaborn as sns
sns.set(rc={'figure.figsize':(12, 10)})
import scipy.stats as stats
from scipy import stats as st
import math

In [4]:
try:
    data = pd.read_csv('/https://code.s3.yandex.net/datasets/logs_exp.csv', sep='\t')
except:    
    data = pd.read_csv('/datasets/logs_exp.csv', sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/logs_exp.csv'

In [ ]:
# Функция для просмотра основной информации о датафрейме
def preview(data):
    display(data.head())
    
    display(data.info())
    
    print('Всего строк:', data.shape[0],', столбцов:', data.shape[1])
    
    'Дубликатов:', data.duplicated().sum()

In [ ]:
preview(data)

<div class="alert alert-block alert-info">
Вывод:
    На первый взгляд, данные полные, не содержат пропусков. Столбцы стоит переименовать для удобства восприятия и информативности. Также видим, что столбец `EventTimestamp` содержит дату и время события - его переведем в формат дата-время. Дополнительно проверим на наличие дублирующихся строк. 
</div>

## Предобработка данных

In [ ]:
# Поменяем названия столбцов для удобства дальнейшей работы
data.rename(columns={'EventName': 'event', 'DeviceIDHash':'user_id', 'EventTimestamp': 'timestamp', 'ExpId': 'group'}, inplace=True)

# Изменим тип данных в колонке 'timestamp'
data['timestamp'] = pd.to_datetime(data.timestamp, unit='s')

# Добавим столбец даты 
data['date'] = data['timestamp'].dt.date
data['date'] = pd.to_datetime(data['date'])

data.head()

In [ ]:
# Проверим на дубликаты
print('Всего дубликатов:', data.duplicated(['event', 'user_id', 'timestamp', 'group', 'date']).sum())
# Удалим дублирующие записи
data = data.drop_duplicates()
print('Дубликатов после обработки:', data.duplicated(['event', 'user_id', 'timestamp', 'group', 'date']).sum())

In [ ]:
data.groupby('user_id')['group'].nunique().value_counts()

    У каждого пользователя только одна группа - и это замечательно!  


In [ ]:
data.info()
print('Контрольная дата, начало периода:', data['date']. min())
print('Контрольная дата, конец периода:', data['date']. max())

**Вывод:**

* В представленном для анализа файле - записи о действиях пользователя и событиях за период с 25.07.2019 по 07.08.2019
* Датафрейм содержит 243713 записей.
* Данные загружены корректно.
* Таблицы не содержат пропусков.
* В датафрейме обнаружены и удалены 413 дублирующих записей.
* Для удобства дальнейшей работы названия столбцов изменены:
    - `event` — название события;
    - `user_id` — уникальный идентификатор пользователя;
    - `timestamp` — время события;
    - `group` — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.
* Добавлен столбец `date`, содержащий дату события.
* Данные, содержащие дату/время, приведены к формату datetime.
* Типы данных соответствуют сохранённым в них значениям.
* Распределение пользователей по тестовым группам корректно.

## Изучение и проверка данных
### Найдем среднее количество событий на пользователя

In [ ]:
# Найдем количество событий в логе
print('Всего событий:', len(data['event']))

# Найдем количество пользователей в логе
print('Уникальных пользователей:', len(data['user_id'].unique()))

# Найдем, сколько в среднем событий приходится на пользователя
print('Среднее количество событий на одного пользователя:', round(data.groupby('user_id')['event'].count().mean(), 2))

In [ ]:
# Проверим, насколько близко среднее значение к медианному
data.groupby('user_id')['event'].count().describe()

Медианное значение составляет 20 событий на пользователя. Присутствует максимальное значение в 2307 событий. Вероятно, это аномалии, в таком случае, в качестве показателя количества событий на пользователя лучше использовать медианное значение.

### Изучим изменение количества данных
Данные представлены за период с 27 июля по 07 августа 2019 г. Построим столбчатую диаграмму, которая отобразит количество событий в зависимости от времени в разрезе групп. 

In [ ]:
data['date'] = data['timestamp'].dt.date

# Создадим сводную таблицу, посчитаем количество событий в каждой группе за каждый день исследования
events_by_dates = data.pivot_table(index='date', 
                  columns ='group', 
                  values='event', aggfunc='count')

# Построим столбчатую диаграмму
events_by_dates.plot(kind='bar', figsize=(10, 7), grid=True)
plt.title('Распределение событий по датам в группах')
plt.xlabel('Дата')
plt.ylabel('Количество событий')
plt.xticks(rotation=90)
plt.show()

In [ ]:
data['date'] = pd.to_datetime(data['date'])

Становится очевидным, что события распределены во времени неравномерно. Слишком мало данных за период с 25 по 31 июля. Значит, для исследования мы возьмем данные с 01 по 07 августа. 

Отбросим неактуальные данные. Узнаем, много ли событий и пользователей мы потеряли. 

In [ ]:
data_filtered = data.query('date >= "2019-08-01"')
print(f'Всего актуальных записей: {len(data_filtered)}')
print(f'Потеряно информации событиях: {1 - (len(data_filtered) / len(data)):.2%}')
print(f'Потеряно информации о пользователях: {1- (len(data_filtered.user_id.unique()) / len(data.user_id.unique())):.2%}')

Проверим, что в нашем исследовании остались пользователи из всех трёх экспериментальных групп.

In [ ]:
data_filtered.groupby('group')['user_id'].nunique()

**Вывод:**
* В исходном датафрейме 243 713 событий и 7 551 уникальный пользователь.
* Среднее количество событий на одного пользователя - 32.28, но учитывая, что в данных присутствуют выбросы, корректней ориентироваться на медианное значение в 20 событий на пользователя.
* События распределены по дням исследования неравномерно. За первую неделю исследования с 25 по 31 июля 2019 года данных слишком мало. Для дальнейшего анализа мы используем данные с 01 по 07 августа, потеряв при этом около 1% информации о событиях и данные о 0,23% пользователей. 
* Пользователи всех трех экспериментальных групп присутсвуют в оставшихся для исследования данных.

## Изучим воронку событий

In [ ]:
# Посмотрим, какие события есть в логах и как часто они встречаются
data_filtered['event'].value_counts()

In [ ]:
# Соберем в таблицу частоту событий и количество пользователей, которые их совершали
event_funnel = data_filtered.groupby('event').agg({'event': 'count', 'user_id':'nunique'}) \
               .rename(columns={'event': 'count_events', 'user_id': 'count_users'}) \
               .sort_values(by='count_events', ascending=False)
                                                  
# Посчитаем долю пользователей, которые хоть раз совершали событие.
event_funnel['share'] = round((event_funnel['count_users'] / data_filtered['user_id'].nunique() * 100), 2)
event_funnel

    Видимо, часть пользователей попадает на страницу каталога, минуя главную страницу, через рассылку или, например, из пуш-уведомлений.


In [ ]:
event_funnel['share'].plot(kind='bar', figsize=(10, 7))
plt.title('Доля пользователей, совершивших событие')
plt.xlabel('Событие')
plt.ylabel('Доля пользователей')
plt.xticks(rotation=45)
plt.show()

Пользователи совершают следующие события:
* MainScreenAppear - просмотр главной страницы
* OffersScreenAppear - просмотр страницы предложений
* CartScreenAppear - просмотр корзины
* PaymentScreenSuccessful - переход на страницу успешной оплаты
* Tutorial - просмотр обучающей информации

На главную страницу приложения попадает 98,8% пользователей. Открывают страницу с предложениями 61% пользователей. В корзину переходит 50% пользователей. На страницу оплаты - 47%. И только 11% просматривают обучающую страницу. Вероятно, просмотр обучающей информации не обязательный этап маршрута пользователя. Зато первые четыре этапа составляют последовательную цепочку движения пользователя. 

По воронке событий посчитаем, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем). 

In [ ]:
# Добавим в таблицу столбец с количеством пользователей на предыдущем шаге
event_funnel['shifted'] = event_funnel['count_users'].shift(1)
# Рассчитаем долю пользователей, перешедших на следующий шаг воронки от предыдущего шага
event_funnel['parth_of_previous'] = round((event_funnel['count_users'] / event_funnel['shifted'] * 100), 2)
# Рассчитаем долю пользователей, перешедших на следующий шаг от общего числа пользователей
event_funnel['parth_of_all'] = round((event_funnel['count_users'] / event_funnel['count_users'][0] * 100), 2)

event_funnel

**Вывод:**
Воронка событий состоит из следующих этапов:
* Просмотр главной страницы - совершает 98% пользователей
* Просмотр страницы предложений - сюда попадает 62% пользователей с предыдущего этапа
* Просмотр корзины - переход на эту страницу совершают 81% пользователей
* Переход на страницу успешной оплаты - этот шаг совершает 95% клиентов из предыдущего шага.

Таком образом, наибольшая потеря пользователей происходит на этапе перехода с главной страницы на страницу предложений. Необходимо передать эту информацию, и проверить возможные ошибки, с которыми могут сталкиваться клиенты. Отметим, что с главной страницы приложения до страницы успешной оплаты доходит 48% пользователей, что является значительным результатом.

## Изучим результаты эксперимента

In [ ]:
# Проверим, сколько пользователей в каждой экспериментальной группе
count_users_in_group = data_filtered.groupby('group')['user_id'].nunique()
count_users_in_group['246+247'] = count_users_in_group[246] + count_users_in_group[247]
count_users_in_group.to_frame()

Пользователи достаточно равномерно разделены на следующие группы:
* 246 контрольная: 2484 пользователя
* 247 контрольная: 2513 пользователей
* 248 экспериментальная: 2537 пользователей
* всего в контрольных группах: 4997 пользователей

Убедимся, что все пользователи каждой группы совершали одни и те же события

In [ ]:
# Подсчитаем, какое количество пользователей в каждой группе совершили каждое событие
# Исключим событие "Просмотр обучающей информации"
group_event = data_filtered.query('event != "Tutorial"').pivot_table(index='event', columns='group', values='user_id', aggfunc='nunique')\
              .sort_values(by=246, ascending=False)

group_event

In [ ]:
group_event.plot(kind='bar', figsize=(10, 7), grid=True)
plt.title('Пользователи, совершившие событие')
plt.xlabel('Событие')
plt.ylabel('Количество пользователей')
plt.xticks(rotation=45)
plt.show()

### Найдем долю пользователей, совершивших каждое событие в каждой группе

In [ ]:
# Сформируем таблицу с количеством пользователей по каждому событию
testing_set = data_filtered.query('event != "Tutorial"')\
    .pivot_table(index='event', columns='group', values='user_id', aggfunc='nunique')\
    .sort_values(by=246, ascending=False).reset_index()

# Добавим столбец суммарных показателей для групп 246 и 247
testing_set['246+247'] = testing_set[246] + testing_set[247]

# Добавим столбец с долей пользователей группы 246, по каждому событию
testing_set['share_246'] = testing_set[246] / count_users_in_group[246]
# Добавим столбец с долей пользователей группы 247, по каждому событию
testing_set['share_247'] = testing_set[247] / count_users_in_group[247]
# Добавим столбец с долей пользователей группы 248, по каждому событию
testing_set['share_248'] = testing_set[248] / count_users_in_group[248]
# Добавим столбец с долей пользователей группы 247, по каждому событию
testing_set['share_246+247'] = testing_set['246+247'] / count_users_in_group['246+247']

testing_set

In [ ]:
from plotly import graph_objects as go


fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Группа 246',
    y = testing_set['event'],
    x = testing_set[246],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Группа 247',
    orientation = "h",
    y = testing_set['event'],
    x = testing_set[247],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Группа 248',
    orientation = "h",
    y = testing_set['event'],
    x = testing_set[248],
    textposition = "inside",
    textinfo = "value+percent initial"))
fig.update_layout(title='Воронка событий по пользователям', title_x = 0.5)
fig.show()   

Доли пользователей, совершивших событие в каждой группе близки, значит распределение событий между экспериментальными группами близко к равномерному, что свидетельствует о корректном разделении данных на группы.

### Выполним А/А-тест

Функция для определения статистической значимости различий между группами

* group_number - номера тестируемых групп
* event - событие, на уровне которого проверяется равенство долей
* alpha - критический уровень статистической значимости

In [ ]:
# Функция test для определения стат.значимости различий между группами
# Принимает на вход датафрейм, номера групп и заданный уровень стат.значимости альфа

def test(df, group_number, alpha):
    
    # Разбиваем пользователей на группы:
    users = [df.query('group == @i')['user_id'].nunique() for i in group_number]
    
    # Перебираем пары групп по событиям и количеству пользователей:    
    for event in df.event.unique():
        events = [df.query('group == %d and event == "%s"' % (group, event))['user_id'].nunique() for group in group_number]
        
        # пропорция успехов в первой группе:
        p1 = events[0] / users[0] 
        
        # пропорция успехов во второй группе:
        p2 = events[1] / users[1] 
        
        # пропорция успехов в комбинированном датасете:
        p_combined = sum(events) / sum(users) 
        
        # разница пропорций в датасетах:
        difference = p1 - p2 
        
        # считаем статистику в ст.отклонениях стандартного нормального распределения:
        z_value = difference / math.sqrt(p_combined * (1 - p_combined) * (1 / users[0] + 1 / users[1]))
        
        # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1):
        distr = st.norm(0, 1)  
        
        p_value = (1 - distr.cdf(abs(z_value))) * 2
        
        print('Событие:', event)
        print('p-значение: {p_value:}'.format(p_value=p_value))
        
        if p_value < alpha:
            print('Отвергаем нулевую гипотезу: между долями есть разница')
        else:
            print('Не отвергаем нулевую гипотезу: различия статистически не значимы')
        print()    

У нас есть 2 контрольные группы для А/А-эксперимента, чтобы проверить корректность всех механизмов и расчётов. Выясним, находят ли статистические критерии разницу между выборками 246 и 247. Так как мы считаем, что эти группы идентичны, то установим точность теста в 1%.

Выдвинем гипотезы:
* Н0 : доли пользователей, совершивших одно и то же событие не отличаются
* Н1 : доли пользователей, совершивших одно и то же событие отличаются

Возьмем уровень значимости альфа равный 1%

In [ ]:
# Событие "просмотр обучающей информации" исключим из тестирования, как маловажное
data_filtered = data_filtered.query('event != "Tutorial"')
test(data_filtered, [246, 247], 0.01)

**Вывод:**

Результаты z-тестов для контрольных групп 246 и 247 не выявили статистически значимых различий между ними.

###  Выполним А/В-тест
Аналогично поступим с группой с изменённым шрифтом. Сравним результаты с каждой из контрольных групп в отдельности по каждому событию. 

**Проведем z-тест для контрольной группы 246 и экспериментальной группой 248.** Для A/B теста не нужна высокая точность, поэтому выберем уровень статистической значимости равный 5%. 

Выдвинем гипотезы:

* Н0 : доли пользователей, совершивших одно и то же событие не отличаются
* Н1 : доли пользователей, совершивших одно и то же событие отличаются

Возьмем уровень значимости альфа равный 5%

In [ ]:
test(data_filtered, [246, 248], 0.05)

**Вывод:**

Результаты z-тестов для контрольной группы 246 и экспериментальной 248 не выявили статистически значимых различий между ними.

**Проведем z-тест для контрольной группы 247 и экспериментальной группой 248.** Выберем уровень статистической значимости равный 5%. 

Выдвинем гипотезы:

* Н0 : доли пользователей, совершивших одно и то же событие не отличаются
* Н1 : доли пользователей, совершивших одно и то же событие отличаются

Возьмем уровень значимости альфа равный 5%

In [ ]:
test(data_filtered, [247, 248], 0.05)

**Вывод:**

Результаты z-тестов для контрольной группы 247 и экспериментальной 248 не выявили статистически значимых различий между ними.

**Проведем z-тест для объединенной контрольной группы и экспериментальной группой 248.**

Выдвинем гипотезы:

* Н0 : доли пользователей, совершивших одно и то же событие не отличаются
* Н1 : доли пользователей, совершивших одно и то же событие отличаются

Возьмем уровень значимости альфа равный 5%

In [ ]:
# создадим таблицу с объединенными данными групп 246 и 247
data_combined = data_filtered.copy()
data_combined['group'].replace({247: 246}, inplace=True)
data_combined.groupby('group')['user_id'].nunique()

In [ ]:
test(data_combined, [246, 248], 0.05)

**Вывод:**

Результаты z-тестов для объединенной контрольной группы и экспериментальной 248 не выявили статистически значимых различий между ними.

Для проверки статистических гипотез мы брали уровень значимости в 0,05. Провели 12 тестов, сравнивая по 4 события в 3 группах. При уровне значимости в 0,1 при проведении в среднем 10% проверок возможно ложное отклонение нулевой гипотезы. Для повышения достоверности результатов мы использовали более низкий уровень значимости. Учитывая, что мы проводили множественные тесты, используя выборки из одного набора данных, присутствует риск увеличения вероятности ошибки первого рода с каждой дополнительной проверкой гипотезы. Мы могли бы скорректировать полученные значения p-value, воспользовавшись методом Бонферрони, уменьшив уровень альфа до 0,05/12=0,00416. Но в нашем случае такой проверки не требуется, так как результаты тестов всех событий всех пар групп выше базового 5% уровня статистической значимости. 

Таким образом, изменение шрифта в приложении не влияет на поведение пользователей.

## Общий вывод

* Мы анализировали работу мобильного приложения для стартапа по продаже продуктов питания.
* Цель исследования заключалась в определении влияния изменения шрифта в приложении на действия пользователей.


* Были реализованы следующие действия:

* Изучен файл с логами о действиях пользователя и событиях за период с 25.07.2019 по 07.08.2019:
    - В исходном датафрейме 243 713 событий и 7 551 уникальный пользователь.
    - Пользователи разспределены на 3 группы равномерно.
    - Среднее количество событий на одного пользователя - 32.28, но учитывая, что в данных присутствуют выбросы, корректней ориентироваться на медианное значение в 20 событий на пользователя.
    - События распределены по дням исследования неравномерно. За первую неделю исследования с 25 по 31 июля 2019 года данных слишком мало. Для дальнейшего исследования мы отсеяли эти данные, оставив для анализа данные с 01 по 07 августа, потеряв при этом около 1% информации о событиях и данные о 0,23% пользователей.
    - Пользователи всех трех экспериментальных групп присутсвуют в оставшихся для исследования данных.


* Пользователи совершают следующие события:
    - MainScreenAppear - просмотр главной страницы
    - OffersScreenAppear - просмотр страницы предложений
    - CartScreenAppear - просмотр корзины
    - PaymentScreenSuccessful - переход на страницу успешной оплаты
    - Tutorial - просмотр обучающей информации.
    
Событие Tutorial не является обязательным этапом пути пользователя.

* Воронка событий состоит из следующих этапов:
    - Просмотр главной страницы - совершает 98% пользователей
    - Просмотр страницы предложений - сюда попадает 62% пользователей с предыдущего этапа
    - Просмотр корзины - переход на эту страницу совершают 81% пользователей
    - Переход на страницу успешной оплаты - этот шаг совершает 95% клиентов из предыдущего шага.
    
Наибольшая потеря пользователей происходит на этапе перехода с главной страницы на страницу предложений. Рекомендовано проверить возможные ошибки, с которыми могут сталкиваться клиенты. Также отмечено, что с главной страницы приложения до страницы успешной оплаты доходит 48% пользователей, что является значительным результатом.


Пользователи достаточно равномерно разделены на следующие группы:
- 246 контрольная: 2484 пользователя
- 247 контрольная: 2513 пользователей
- 248 экспериментальная: 2537 пользователей

Все пользователи каждой группы совершали одни и те же события.
Доли пользователей, совершивших событие в каждой группе близки, значит распределение событий между экспериментальными группами близко к равномерному, что свидетельствует о корректном разделении данных на группы.


* Для оценки влияния изменений на пользовательские группы была проведена серия А/А тестов на контрольных группах 246 и 247 с уровнем статистической значимости в 1%. В контрольных группах шрифты остались прежними, в эксериментальной - измененными. По результатам этих тестов, статистически значимых различий между двумя контрольными группами не найдено. Это подтверждает корректность распределения пользователей на группы для исследования.


* Аналогичным образом была проведена серия А/В-тестов, где контрольные группы сравнивались с экспериментальной по отдельности и затем обе вместе на каждом шаге воронки событий. Были сформированы нулевая и альтернативная гипотезы:
* Н0 : доли пользователей, совершивших одно и то же событие не отличаются;
* Н1 : доли пользователей, совершивших одно и то же событие отличаются.

В результате проведения 12 А/В-тестов, статистически значимых различий между контрольными и экспериментальной группами не выявлено ни на одном этапе, а значит нельзя утверждать, что изменение шрифтов в приложении отразилось на поведении пользователей. 